In [1]:
import tensorflow_datasets as tfds

/home/polivares/anaconda3/envs/DataScience/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

2022-06-16 22:42:07.247668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 22:42:07.319731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 22:42:07.319906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 22:42:07.320522: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [3]:
test_set, valid_set, train_set = tfds.load("tf_flowers",split=["train[0%:10%]", "train[10%:25%]", "train[25%:]"], as_supervised=True )

In [4]:
import tensorflow as tf

In [5]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [6]:
batch_size = 32
train_set = train_set.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)

In [7]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(5, activation="softmax")(avg)

model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [8]:
for layer in base_model.layers:
    layer.trainable = False # Esto impide que las capas se re entrenen

In [11]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", 
              metrics=["accuracy"])

In [12]:
history = model.fit(train_set, epochs=10, validation_data=valid_set)

Epoch 1/10


2022-06-16 22:42:28.764341: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300


 2/86 [..............................] - ETA: 6s - loss: 1.9047 - accuracy: 0.1875  

2022-06-16 22:42:30.475823: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.47GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-06-16 22:42:30.475866: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.47GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-06-16 22:42:30.540539: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


86/86 [==============================] - 13s 99ms/step - loss: 1.0022 - accuracy: 0.6886 - val_loss: 0.7132 - val_accuracy: 0.7858
Epoch 2/10
86/86 [==============================] - 8s 90ms/step - loss: 0.6022 - accuracy: 0.8252 - val_loss: 0.5878 - val_accuracy: 0.8022
Epoch 3/10
86/86 [==============================] - 8s 90ms/step - loss: 0.5063 - accuracy: 0.8477 - val_loss: 0.5358 - val_accuracy: 0.8076
Epoch 4/10
86/86 [==============================] - 8s 90ms/step - loss: 0.4563 - accuracy: 0.8583 - val_loss: 0.5018 - val_accuracy: 0.8348
Epoch 5/10
86/86 [==============================] - 8s 92ms/step - loss: 0.4221 - accuracy: 0.8688 - val_loss: 0.4792 - val_accuracy: 0.8421
Epoch 6/10
86/86 [==============================] - 8s 93ms/step - loss: 0.3964 - accuracy: 0.8783 - val_loss: 0.4619 - val_accuracy: 0.8457
Epoch 7/10
86/86 [==============================] - 8s 94ms/step - loss: 0.3773 - accuracy: 0.8812 - val_loss: 0.4461 - val_accuracy: 0.8475
Epoch 8/10
86/86 [=====

In [13]:
model.predict(test_set)

array([[7.0090086e-04, 2.0476116e-04, 9.8758775e-01, 1.5770533e-03,
        9.9294288e-03],
       [5.1078450e-02, 1.8542103e-01, 3.5324737e-01, 3.5876185e-01,
        5.1491380e-02],
       [2.1344449e-01, 2.9448453e-02, 5.5590451e-02, 6.7699671e-01,
        2.4519885e-02],
       ...,
       [9.9687648e-01, 3.9770352e-04, 9.9209126e-04, 1.4842463e-03,
        2.4940015e-04],
       [3.8003606e-01, 9.3785807e-02, 7.8692429e-02, 3.3800596e-01,
        1.0947971e-01],
       [9.6869409e-02, 1.3345393e-02, 5.8264881e-03, 8.7802172e-01,
        5.9369928e-03]], dtype=float32)

In [14]:
for image, label in test_set:
    print("Etiqueta real",label)
    print("Predicción", model.predict(image))


Etiqueta real tf.Tensor([2 3 3 4 3 0 0 0 0 1 3 2 4 1 2 1 2 4 2 2 0 0 0 2 0 3 0 1 1 1 2 1], shape=(32,), dtype=int64)
Predicción [[7.00900855e-04 2.04761163e-04 9.87587750e-01 1.57705334e-03
  9.92942881e-03]
 [5.10784499e-02 1.85421035e-01 3.53247374e-01 3.58761847e-01
  5.14913797e-02]
 [2.13444486e-01 2.94484533e-02 5.55904508e-02 6.76996708e-01
  2.45198850e-02]
 [1.16197854e-01 4.67261784e-02 3.30866367e-01 3.63204509e-01
  1.43005058e-01]
 [8.96890163e-02 1.32256257e-03 3.01705743e-03 9.05237317e-01
  7.34017405e-04]
 [8.21522415e-01 9.38070193e-02 1.65027939e-02 6.71728253e-02
  9.95032722e-04]
 [8.88944924e-01 4.98062111e-02 2.71345936e-02 2.91923676e-02
  4.92190011e-03]
 [9.47224915e-01 1.64036676e-02 7.87378289e-03 7.99729675e-03
  2.05003023e-02]
 [9.88866866e-01 3.03149107e-03 3.99237825e-03 3.31603736e-03
  7.93275656e-04]
 [4.05229926e-02 8.11050773e-01 5.64703383e-02 8.67144465e-02
  5.24146063e-03]
 [1.47872314e-01 1.00403078e-01 2.38564402e-01 2.39508331e-01
  2.736518

Predicción [[9.71988440e-01 2.16722046e-03 8.48480873e-03 1.47167416e-02
  2.64274934e-03]
 [1.97612606e-02 1.49281053e-02 9.07163620e-01 4.98771220e-02
  8.26997031e-03]
 [9.09757078e-01 9.29404981e-03 1.99477244e-02 2.80925557e-02
  3.29085402e-02]
 [2.55084615e-02 1.23580853e-02 9.55012143e-01 3.04293423e-03
  4.07843851e-03]
 [5.47353039e-03 9.71270539e-03 7.25886375e-02 2.57193595e-01
  6.55031443e-01]
 [7.81966024e-04 3.66398762e-03 2.28191577e-02 1.53855316e-03
  9.71196294e-01]
 [9.01299298e-01 8.14475026e-03 1.17355343e-02 7.15602711e-02
  7.26019125e-03]
 [1.14761619e-02 6.85692355e-02 4.68719471e-03 9.13793981e-01
  1.47342077e-03]
 [8.54296148e-01 1.07324965e-01 3.37066571e-03 3.36005203e-02
  1.40776357e-03]
 [6.95721447e-01 2.12406069e-01 3.44631192e-03 8.74507129e-02
  9.75398580e-04]
 [1.49514270e-03 9.93698180e-01 2.28157791e-04 4.28756094e-03
  2.90878845e-04]
 [1.58968307e-02 7.32675521e-03 9.12993193e-01 4.76233196e-03
  5.90209402e-02]
 [6.51905596e-01 1.12452865e-

Predicción [[1.00296328e-03 9.98446524e-01 2.74912483e-04 2.53881270e-04
  2.16148401e-05]
 [6.86073583e-03 1.83844951e-03 9.82728839e-01 1.87858986e-03
  6.69327378e-03]
 [4.80521889e-03 2.90942402e-03 9.41368699e-01 4.35337909e-02
  7.38287857e-03]
 [3.16591642e-04 3.23253917e-04 6.42344169e-03 2.70131766e-03
  9.90235388e-01]
 [1.63143559e-04 9.98536706e-01 7.97814355e-05 1.18284358e-03
  3.75480740e-05]
 [8.89686111e-04 1.12761231e-03 6.60269856e-01 2.05601994e-02
  3.17152649e-01]
 [2.55632252e-01 4.69201012e-03 2.76057012e-02 6.37914717e-01
  7.41553456e-02]
 [2.68432889e-02 3.19849746e-03 8.27484727e-02 8.66396606e-01
  2.08130982e-02]
 [9.80418861e-01 8.29296745e-03 6.26974227e-03 2.52763997e-03
  2.49072723e-03]
 [9.92253482e-01 7.77492067e-04 2.07350776e-03 4.30713408e-03
  5.88334573e-04]
 [9.96496379e-01 3.00988118e-04 3.34262440e-04 2.41761049e-03
  4.50674532e-04]
 [1.21336379e-04 3.27716931e-04 9.87130702e-01 4.50362102e-04
  1.19699733e-02]
 [1.32029057e-01 1.31251261e-